# Random Forest survival

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest

set_config(display="text")  # displays text representation of estimators

In [9]:
import pandas as pd
df = pd.read_csv('miRNAexpression_clinicle.csv')
df.head()

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-944,hsa-mir-95,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b,Diagnosisage,LymphNodeStage,Survival,SurvivalStatus
0,12.5118,13.5062,12.5058,13.1423,9.4597,9.1296,11.3277,4.5361,13.1852,8.7630,...,7.0844,4.4115,3.2474,5.8341,7.7704,14.8284,66.0,2.0,3.353388,0
1,12.8151,13.8253,12.8277,13.0250,8.0617,9.6969,9.6906,4.6816,14.1684,9.4817,...,9.5068,1.5428,5.5523,6.5335,5.1035,14.0989,47.0,2.0,12.788901,1
2,13.9389,14.9395,13.9522,14.2588,8.8684,9.0935,10.8985,5.9200,14.9715,10.4961,...,8.7194,4.2529,4.5285,6.3210,4.7599,14.2215,56.0,0.0,42.344741,0
3,13.7992,14.8060,13.8097,14.8904,11.5533,9.5417,9.7590,4.8165,13.8766,9.4976,...,7.5526,2.5631,5.6962,6.5749,8.7677,13.6794,54.0,2.0,8.153335,0
4,13.6592,14.6622,13.6761,14.4914,9.4091,9.2256,9.4294,6.3407,13.8757,9.8671,...,7.6184,2.6753,4.1333,5.6403,6.3945,13.7741,58.0,0.0,23.835355,0


In [10]:
df=df.fillna(value=0)
df.isnull().sum()
df2=np.array(df)
# find min and max values for each column, ignoring nan, -inf, and inf
mins = [np.nanmin(df2[:, i][df2[:, i] != -np.inf]) for i in range(df2.shape[1])]
maxs = [np.nanmax(df2[:, i][df2[:, i] != np.inf]) for i in range(df2.shape[1])]

#df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
for i in range(df.shape[1]):
    df2[:, i][df2[:, i] == -np.inf] = mins[i]
    df2[:, i][df2[:, i] == np.inf] = maxs[i]

In [12]:
df2=pd.DataFrame(df2)
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,836,837,838,839,840,841,842,843,844,845
0,12.5118,13.5062,12.5058,13.1423,9.4597,9.1296,11.3277,4.5361,13.1852,8.7630,...,7.0844,4.4115,3.2474,5.8341,7.7704,14.8284,66.0,2.0,3.353388,0.0
1,12.8151,13.8253,12.8277,13.0250,8.0617,9.6969,9.6906,4.6816,14.1684,9.4817,...,9.5068,1.5428,5.5523,6.5335,5.1035,14.0989,47.0,2.0,12.788901,1.0
2,13.9389,14.9395,13.9522,14.2588,8.8684,9.0935,10.8985,5.9200,14.9715,10.4961,...,8.7194,4.2529,4.5285,6.3210,4.7599,14.2215,56.0,0.0,42.344741,0.0
3,13.7992,14.8060,13.8097,14.8904,11.5533,9.5417,9.7590,4.8165,13.8766,9.4976,...,7.5526,2.5631,5.6962,6.5749,8.7677,13.6794,54.0,2.0,8.153335,0.0
4,13.6592,14.6622,13.6761,14.4914,9.4091,9.2256,9.4294,6.3407,13.8757,9.8671,...,7.6184,2.6753,4.1333,5.6403,6.3945,13.7741,58.0,0.0,23.835355,0.0


In [13]:
Y2=df2[df2.columns[845:]].astype(bool)
Y2=pd.DataFrame(Y2)

Y3=df2[df2.columns[844]]
Y3=pd.DataFrame(Y3)

Y3=Y3.drop(Y3[Y3.columns[845:]], axis=1)
#print(Y3)
Y4=pd.concat([Y2,Y3], axis=1)
Y4


,845,844
0,False,3.353388
1,True,12.788901
2,False,42.344741
3,False,8.153335
4,False,23.835355
...,...,...
478,True,17.062827
479,True,23.605221
480,True,1.808199
481,False,17.029950


In [14]:
y = Y4.to_records(index=False)
y

rec.array([(False, 3.35338791e+00), ( True, 1.27889009e+01),
           (False, 4.23447414e+01), (False, 8.15333531e+00),
           (False, 2.38353552e+01), (False, 2.04819673e+01),
           (False, 6.55883223e+01), ( True, 5.30624322e+01),
           ( True, 1.13094651e+01), (False, 2.11723707e+01),
           ( True, 1.48272348e+01), ( True, 3.55064602e+00),
           ( True, 1.26245192e+01), (False, 2.16983923e+00),
           (False, 4.18515961e+01), (False, 1.52217510e+01),
           (False, 2.81092810e+01), (False, 2.04490910e+01),
           (False, 6.47664135e+00), (False, 2.79120229e+01),
           (False, 1.15395996e+01), (False, 6.83828122e+00),
           ( True, 3.15612980e+00), (False, 1.44655949e+01),
           ( True, 7.82457179e+00), ( True, 2.26846829e+00),
           ( True, 8.74510964e+00), ( True, 7.89032449e+00),
           (False, 2.04819673e+01), (False, 1.64052997e+01),
           (False, 6.14787783e+00), (False, 2.79448992e+00),
           (False, 2.242

In [15]:
#new_df=df.drop(labels=[ 'Survival',  'SurvivalStatus' ], axis=1)
new_df=df2.drop((df2[df2.columns[844:]]), axis=1)
count = np.isinf(new_df).values.sum()
count
new_df.head()

,0,1,2,3,4,5,6,7,8,9,...,834,835,836,837,838,839,840,841,842,843
0,12.5118,13.5062,12.5058,13.1423,9.4597,9.1296,11.3277,4.5361,13.1852,8.7630,...,2.4290,0.0000,7.0844,4.4115,3.2474,5.8341,7.7704,14.8284,66.0,2.0
1,12.8151,13.8253,12.8277,13.0250,8.0617,9.6969,9.6906,4.6816,14.1684,9.4817,...,4.3008,0.2781,9.5068,1.5428,5.5523,6.5335,5.1035,14.0989,47.0,2.0
2,13.9389,14.9395,13.9522,14.2588,8.8684,9.0935,10.8985,5.9200,14.9715,10.4961,...,4.1204,0.0000,8.7194,4.2529,4.5285,6.3210,4.7599,14.2215,56.0,0.0
3,13.7992,14.8060,13.8097,14.8904,11.5533,9.5417,9.7590,4.8165,13.8766,9.4976,...,3.2618,0.0000,7.5526,2.5631,5.6962,6.5749,8.7677,13.6794,54.0,2.0
4,13.6592,14.6622,13.6761,14.4914,9.4091,9.2256,9.4294,6.3407,13.8757,9.8671,...,4.7225,0.0000,7.6184,2.6753,4.1333,5.6403,6.3945,13.7741,58.0,0.0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size=0.15)


In [17]:
rsf = RandomSurvivalForest(
    n_estimators=100, min_samples_split=10, min_samples_leaf=15, n_jobs=-1
)
rsf.fit(X_train, y_train)

RandomSurvivalForest(min_samples_leaf=15, min_samples_split=10, n_jobs=-1)

# concordance index 

In [15]:
rsf.score(X_test,y_test)


0.5708639186900056

In [17]:
X_test_sorted = X_test.sort_values(by=[1])
X_test_sel = pd.concat((X_test_sorted.head(3), X_test_sorted.tail(3)))

X_test_sel


,0,1,2,3,4,5,6,7,8,9,...,834,835,836,837,838,839,840,841,842,843
34,11.2513,12.2117,11.3254,13.6401,9.1031,8.4919,7.6285,2.9907,10.3702,8.1118,...,2.9907,0.0,7.0044,1.7295,4.1055,4.6622,7.7025,13.6500,43.0,0.0
292,11.2864,12.2899,11.2944,13.0552,7.5409,7.4559,7.1930,3.8176,10.9249,8.5168,...,3.5811,0.0,7.2432,2.6968,5.1591,4.6478,5.8798,12.2006,57.0,0.0
335,11.6078,12.5712,11.6422,13.3376,8.0074,8.6197,9.0363,3.5379,11.5501,8.2297,...,3.7211,0.0,7.8194,3.6324,3.8046,5.8483,6.3364,14.0574,59.0,2.0
10,14.0364,15.0486,14.0442,14.3946,10.8365,9.2623,10.7628,5.8940,14.8552,10.5773,...,3.9575,0.0,8.0650,3.5828,4.0727,6.3302,7.9378,13.8835,60.0,0.0
289,14.3314,15.3441,14.3414,14.6255,10.7235,9.1787,12.0917,4.7945,14.8020,9.7933,...,3.8616,0.0,6.1680,1.8393,3.7623,7.3696,7.6841,14.2296,38.0,0.0
147,14.6422,15.6411,14.6398,14.5085,11.2665,9.3130,11.7643,5.2423,16.1545,10.5665,...,3.8708,0.0,7.5645,3.2603,3.0996,6.2133,7.9296,13.1334,62.0,0.0


In [18]:
pd.Series(rsf.predict(X_test_sel))


0    57.681446
1    44.554907
2    60.083784
3    48.932498
4    59.233230
5    37.045513
dtype: float64

# Permutation-based Feature Importance

In [19]:
from sklearn.inspection import permutation_importance

result = permutation_importance(rsf, X_test, y_test, n_repeats=15)

In [20]:
pd.DataFrame(
    {
        k: result[k]
        for k in (
            "importances_mean",
            "importances_std",
        )
    },
    index=X_test.columns,
).sort_values(by="importances_mean", ascending=False)

,importances_mean,importances_std
585,0.007303,0.004089
589,0.005402,0.003590
428,0.005289,0.001334
400,0.005195,0.001982
668,0.004705,0.001952
...,...,...
69,-0.004668,0.002452
660,-0.004837,0.001443
842,-0.005327,0.008754
68,-0.005439,0.004968


In [21]:
# Gradient Boosted Models

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

from sklearn.model_selection import train_test_split
from sksurv.datasets import load_breast_cancer
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder

In [24]:
est_cph_tree = GradientBoostingSurvivalAnalysis(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
est_cph_tree.fit(X_train, y_train)
cindex = est_cph_tree.score(X_test, y_test)
print(round(cindex, 3))

0.622


In [25]:
pd.Series(est_cph_tree.predict(X_test_sel))


0   -0.342643
1   -1.087762
2   -0.268763
3    0.436549
4    3.598268
5    1.092798
dtype: float64

In [ ]:
CoX Regression

In [27]:
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis

cph = CoxPHSurvivalAnalysis()
cph.fit(X_train, y_train)


C:\Users\Lenovo\anaconda3\envs\Hover-net\lib\site-packages\sksurv\linear_model\coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=1.00899e-23): result may not be accurate.
  delta = solve(
C:\Users\Lenovo\anaconda3\envs\Hover-net\lib\site-packages\sksurv\linear_model\coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=3.29883e-23): result may not be accurate.
  delta = solve(
C:\Users\Lenovo\anaconda3\envs\Hover-net\lib\site-packages\sksurv\linear_model\coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=1.83082e-23): result may not be accurate.
  delta = solve(
C:\Users\Lenovo\anaconda3\envs\Hover-net\lib\site-packages\sksurv\linear_model\coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=4.0798e-24): result may not be accurate.
  delta = solve(
C:\Users\Lenovo\anaconda3\envs\Hover-net\lib\site-packages\sksurv\linear_model\coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=8.78649e-24): result may not be accurate.
  delta = solve(
C:\Users\Lenovo\

ValueError: search direction contains NaN or infinite values

In [ ]:
pd.Series(cph.predict(X_test_sel))

# Lungs Cancer

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest

set_config(display="text")  # displays text representation of estimators

In [7]:
df=df.fillna(value=0)
df.isnull().sum()
df2=np.array(df)
# find min and max values for each column, ignoring nan, -inf, and inf
mins = [np.nanmin(df2[:, i][df2[:, i] != -np.inf]) for i in range(df2.shape[1])]
maxs = [np.nanmax(df2[:, i][df2[:, i] != np.inf]) for i in range(df2.shape[1])]

#df.replace([np.inf, -np.inf], np.nan, inplace=True)

NameError: name 'df' is not defined

In [ ]:
for i in range(df.shape[1]):
    df2[:, i][df2[:, i] == -np.inf] = mins[i]
    df2[:, i][df2[:, i] == np.inf] = maxs[i]

In [ ]:
df2=pd.DataFrame(df2)
df2.head()

In [ ]:
Y2=df2[df2.columns[845:]].astype(bool)
Y2=pd.DataFrame(Y2)

Y3=df2[df2.columns[844]]
Y3=pd.DataFrame(Y3)

Y3=Y3.drop(Y3[Y3.columns[845:]], axis=1)
#print(Y3)
Y4=pd.concat([Y2,Y3], axis=1)
Y4


In [ ]:
y = Y4.to_records(index=False)
y

In [ ]:
#new_df=df.drop(labels=[ 'Survival',  'SurvivalStatus' ], axis=1)
new_df=df2.drop((df2[df2.columns[844:]]), axis=1)
count = np.isinf(new_df).values.sum()
count
new_df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size=0.15)

In [ ]:
rsf = RandomSurvivalForest(
    n_estimators=100, min_samples_split=10, min_samples_leaf=15, n_jobs=-1
)
rsf.fit(X_train, y_train)

In [ ]:
rsf.score(X_test,y_test)